Drag this into the _data folder or set directories for the parquet files

In [2]:
import pandas as pd
import numpy as np

CDS1 = pd.read_parquet("markit_cds_1.parquet")
CDS2 = pd.read_parquet("markit_cds_2.parquet")
bonds = pd.read_parquet("mergent_bond.parquet")
mapping = pd.read_parquet("RED_and_ISIN_mapping.parquet")
wbonds = pd.read_parquet("wrds_bond.parquet")

In [3]:
CDSs = pd.concat([CDS1, CDS2], ignore_index=True)

In [4]:
mapping.dropna(inplace=True)
mapping["issuer_cusip"] = mapping.apply(lambda row: row['obl_cusip'][:6], axis = 1)	
mapping

,redcode,ticker,obl_cusip,isin,tier,issuer_cusip
0,001AEC,T,001957AV1,US001957AV12,SNRFOR,001957
1,001AEC,T,001957BC2,US001957BC22,SNRFOR,001957
2,001AEC,T,001957AP4,US001957AP44,SNRFOR,001957
3,001AEC,T,001957BD0,US001957BD05,SNRFOR,001957
4,002AA6,ABT,002824AK6,US002824AK63,SNRFOR,002824
...,...,...,...,...,...,...
10377,YZ79AI,KAWAH,Y1636YAG3,XS0205831851,SNRFOR,Y1636Y
10378,YZ79CL,CITPAC-Fin,G2154WAA0,USG2154WAA02,SNRFOR,G2154W
10384,YZ7C7O,CMCOMP,G2233YAA1,USG2233YAA13,SNRFOR,G2233Y
10385,YZ9A6N,CTBCFIN,DUMYREFOB,XSNOREFOBL00,SNRFOR,DUMYRE


<span style="color: Cyan; font-size: 35px">
USING NEW BONDS TABLE
</span>

In [5]:
wbonds = pd.read_parquet("wrds_bond.parquet")
wbonds

,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,principal_amt,maturity_time_frame,rating
date,,,,,,,,,,,,,,,
2002-07-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.048266,000361AB1,US000361AB18,0.0,100.0,102.791000,NaN,1000.0,10,1
2002-08-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.043858,000361AB1,US000361AB18,0.0,100.0,103.089000,NaN,1000.0,10,1
2002-09-30,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.041222,000361AB1,US000361AB18,0.0,100.0,103.143000,NaN,1000.0,10,1
2002-10-31,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.038728,000361AB1,US000361AB18,0.0,100.0,103.143000,NaN,1000.0,10,1
2002-11-30,1993-10-12,AIR,2003-10-15,50000.0,SEN,0.040151,000361AB1,US000361AB18,0.0,100.0,102.750000,0.034101,1000.0,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,2024-03-25,BCS,2026-03-27,13748.0,SEN,0.003740,06745Q3E7,US06745Q3E77,0.0,100.0,99.250000,NaN,1000.0,2,1
2024-03-31,2024-03-25,BCS,2029-03-29,11765.0,SEN,0.003428,06748L453,US06748L4538,0.0,100.0,9.828117,NaN,10.0,5,1
2024-04-30,2024-03-25,BCS,2029-03-29,11765.0,SEN,0.003485,06748L453,US06748L4538,0.0,100.0,9.828117,NaN,10.0,5,1


In [6]:
redcode_mapping = mapping.set_index("obl_cusip")["redcode"].to_dict()

wbonds["redcode"] = wbonds["cusip"].map(redcode_mapping)
wbonds = wbonds.dropna(subset=['redcode', 'yield','price_eom', "cusip", "isin"])

wbonds.sort_index()

wbonds

,offering_date,company_symbol,maturity,amount_outstanding,security_level,yield,cusip,isin,conv,offering_price,price_eom,t_spread,principal_amt,maturity_time_frame,rating,redcode
date,,,,,,,,,,,,,,,,
2002-07-31,1995-02-16,WYE,2005-02-15,1000000.0,SEN,0.052477,026609AE7,US026609AE79,0.0,99.805,106.262285,0.013846,1000.0,9,0,9HB326
2002-08-31,1995-02-16,WYE,2005-02-15,1000000.0,SEN,0.040351,026609AE7,US026609AE79,0.0,99.805,108.930922,0.006612,1000.0,9,0,9HB326
2002-09-30,1995-02-16,WYE,2005-02-15,1000000.0,SEN,0.035810,026609AE7,US026609AE79,0.0,99.805,109.723387,0.007890,1000.0,9,0,9HB326
2002-10-31,1995-02-16,WYE,2005-02-15,1000000.0,SEN,0.036073,026609AE7,US026609AE79,0.0,99.805,109.325787,0.004066,1000.0,9,0,9HB326
2002-11-30,1995-02-16,WYE,2005-02-15,1000000.0,SEN,0.037400,026609AE7,US026609AE79,0.0,99.805,108.708000,0.004480,1000.0,9,0,9HB326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,2024-02-15,INTC,2034-02-21,900000.0,SEN,0.050937,458140CL2,US458140CL20,0.0,99.706,100.434590,0.000987,1000.0,10,0,49D954
2024-02-29,2024-02-22,CAT,2029-02-27,600000.0,SEN,0.048081,14913UAJ9,US14913UAJ97,0.0,99.864,100.190175,0.000793,1000.0,5,0,15DA2S
2024-03-31,2024-02-22,CAT,2029-02-27,600000.0,SEN,0.046884,14913UAJ9,US14913UAJ97,0.0,99.864,100.703493,0.000852,1000.0,5,0,15DA2S


In [7]:
wbonds.loc[~((wbonds['yield'] > 0) & (wbonds['yield'] < 0.5))] = np.nan
wbonds.dropna(subset = "yield", inplace = True)
wbonds = wbonds[wbonds['amount_outstanding'] >= 100000]


C:\Users\vugvu\AppData\Local\Temp\ipykernel_9060\3355583353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wbonds.dropna(subset = "yield", inplace = True)


In [8]:
wbonds['year'] = pd.to_datetime(wbonds.index).year

red_code_dict = {}

for y in list(wbonds['year'].unique()):
    y_df = wbonds[wbonds['year'] == y]
    red_code_dict[y] = list(y_df['redcode'].unique())

C:\Users\vugvu\AppData\Local\Temp\ipykernel_9060\594307929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wbonds['year'] = pd.to_datetime(wbonds.index).year


In [11]:
CDSs.dropna(subset=["parspread"], inplace=True)

In [12]:
bond_average = wbonds.groupby([wbonds.index, 'redcode','maturity_time_frame'])['yield'].mean()

bond_average

date        redcode  maturity_time_frame
2002-07-31  002AA6   10.0                   0.047273
            003CB6   10.0                   0.078488
            014B98   10.0                   0.053955
            018A97   4.0                    0.114206
            018A99   9.0                    0.091530
                                              ...   
2024-03-31  UU2679   8.0                    0.069602
                     9.0                    0.065308
            UW8EGE   10.0                   0.064487
            Y07CDL   9.0                    0.055996
2024-04-30  1D8745   10.0                   0.047623
Name: yield, Length: 176038, dtype: float64

In [13]:
CDSs['tenor'] = CDSs.loc[:,"tenor"].str.split('Y').str.join('').astype(int)


In [14]:
CDSs['date'].describe()

count                         45271738
mean     2013-03-23 00:09:40.109738752
min                2002-01-01 00:00:00
25%                2008-07-30 00:00:00
50%                2013-06-19 00:00:00
75%                2017-09-27 00:00:00
max                2023-12-29 00:00:00
Name: date, dtype: object

In [15]:
group_average = CDSs.groupby(['date', 'redcode'])['parspread'].mean()

group_average

date        redcode
2002-01-01  002AA6     0.002087
            014B98     0.003625
            027A8A     0.014470
            027D97     0.007076
            03AABH     0.007925
                         ...   
2023-12-29  9J127M     0.003178
            9J26DD     0.006866
            UU2679     0.037908
            UW8EGE     0.008070
            UZDD52     0.004111
Name: parspread, Length: 2304175, dtype: float64

In [16]:
cds_spread_df = group_average.to_frame()
bond_spread_df = bond_average.to_frame()

In [17]:
bond_spread_df

yield
date       redcode maturity_time_frame          
2002-07-31 002AA6  10.0                 0.047273
           003CB6  10.0                 0.078488
           014B98  10.0                 0.053955
           018A97  4.0                  0.114206
           018A99  9.0                  0.091530
...                                          ...
2024-03-31 UU2679  8.0                  0.069602
                   9.0                  0.065308
           UW8EGE  10.0                 0.064487
           Y07CDL  9.0                  0.055996
2024-04-30 1D8745  10.0                 0.047623

[176038 rows x 1 columns]

In [18]:
cds_spread_df

parspread
date       redcode           
2002-01-01 002AA6    0.002087
           014B98    0.003625
           027A8A    0.014470
           027D97    0.007076
           03AABH    0.007925
...                       ...
2023-12-29 9J127M    0.003178
           9J26DD    0.006866
           UU2679    0.037908
           UW8EGE    0.008070
           UZDD52    0.004111

[2304175 rows x 1 columns]

In [19]:
joined_df = bond_spread_df.join(cds_spread_df, how='inner')


In [20]:
joined_df

yield  parspread
date       redcode maturity_time_frame                     
2002-07-31 002AA6  10.0                 0.047273   0.002915
           014B98  10.0                 0.053955   0.003330
           018A97  4.0                  0.114206   0.050707
           018A99  9.0                  0.091530   0.058264
           019G83  9.0                  0.097245   0.015000
...                                          ...        ...
2023-11-30 9J26DD  10.0                 0.054099   0.006750
           UU2679  8.0                  0.082843   0.042013
                   9.0                  0.076756   0.042013
           UW8EGE  10.0                 0.070404   0.011155
           UZDD52  10.0                 0.060654   0.004287

[92751 rows x 2 columns]

In [21]:
import pickle
with open('cds_final.pkl', 'wb') as file:
    pickle.dump(CDSs, file)

In [22]:
with open('bond_cds_combined.pkl', 'wb') as file:
    pickle.dump(joined_df, file)

In [ ]:
classification = wbonds[["redcode","rating"]]
df1_reset = joined_df.reset_index()

# Merge on the 'first' column
merged_df = pd.merge(df1_reset, classification, on=['redcode',"date"], how='right')

In [ ]:
merged_df.dropna(inplace=True)

In [ ]:
merged_df.set_index(['date'], inplace= True)

merged_df.sort_index(inplace=True)
merged_df.reset_index(inplace=True)
merged_df.set_index(['date', 'redcode'], inplace= True)
merged_df

yield  parspread  rating
date       redcode                             
2002-07-31 9HB326   0.052477   0.021188     0.0
           39FF84   0.057265   0.006548     1.0
           5H99BW   0.210971   0.282787     1.0
           6FGGC6   0.087077   0.029000     1.0
           3E56A5   0.058903   0.005960     1.0
...                      ...        ...     ...
2023-11-30 3J758C   0.056651   0.015680     0.0
           2C033N   0.053004   0.004853     0.0
           49EB20   0.050556   0.004341     0.0
           2F664B   0.054276   0.005773     0.0
           3H994Q   0.065285   0.019879     0.0

[118984 rows x 3 columns]

In [ ]:
rows_with_A = merged_df.xs('2002-07-31', level='date')
print(rows_with_A)

            yield  parspread  rating
redcode                             
9HB326   0.052477   0.021188     0.0
39FF84   0.057265   0.006548     1.0
5H99BW   0.210971   0.282787     1.0
6FGGC6   0.087077   0.029000     1.0
3E56A5   0.058903   0.005960     1.0
...           ...        ...     ...
8GA758   0.044442   0.005250     1.0
7BDA4H   0.255778   0.385672     1.0
5A788N   0.058801   0.021344     1.0
5I759F   0.060080   0.008000     1.0
6E86CB   0.079560   0.018080     1.0

[328 rows x 3 columns]


In [ ]:
duplicates = rows_with_A.duplicated()
print("Duplicates (excluding first occurrence):")
print(duplicates)

# To see all duplicates (marking every occurrence as True), use keep=False
all_duplicates = rows_with_A.duplicated(keep=False)
print("\nAll duplicates (every duplicate row marked True):")
print(all_duplicates)

# Get a DataFrame containing only the duplicate rows
duplicate_rows = rows_with_A[rows_with_A.duplicated(keep=False)]
print("\nDuplicate rows:")
print(duplicate_rows)

# Check if there are any full duplicate rows in the DataFrame
has_duplicates = rows_with_A.duplicated().any()
print("\nAre there any duplicate rows?", has_duplicates)

Duplicates (excluding first occurrence):
redcode
9HB326    False
39FF84    False
5H99BW    False
6FGGC6    False
3E56A5    False
          ...  
8GA758    False
7BDA4H     True
5A788N     True
5I759F    False
6E86CB    False
Length: 328, dtype: bool

All duplicates (every duplicate row marked True):
redcode
9HB326    False
39FF84     True
5H99BW    False
6FGGC6    False
3E56A5    False
          ...  
8GA758    False
7BDA4H     True
5A788N     True
5I759F    False
6E86CB    False
Length: 328, dtype: bool

Duplicate rows:
            yield  parspread  rating
redcode                             
39FF84   0.057265   0.006548     1.0
0D874B   0.051936   0.002447     1.0
8FA47E   0.080355   0.041664     0.0
189BFD   0.057217   0.008604     1.0
8AAG87   0.057414   0.010444     1.0
...           ...        ...     ...
1E786B   0.050635   0.004830     1.0
59DDA9   0.198244   0.191104     1.0
279AEF   0.054620   0.011750     1.0
7BDA4H   0.255778   0.385672     1.0
5A788N   0.058801   0.021344 